# uITS3 clusters
+ Notebook for visualizing clusters measured with the uITS3 setup in Krakow 22 testbeam

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1">Import libraries</a></span></li><li><span><a href="#Read-clusters-csv-file" data-toc-modified-id="Read-clusters-csv-file-2">Read clusters csv file</a></span></li><li><span><a href="#Define-dashboard" data-toc-modified-id="Define-dashboard-3">Define dashboard</a></span></li></ul></div>

### Import libraries
+ Using RootInteractive https://github.com/miranov25/RootInteractive

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehInteractiveParameters import *
from RootInteractive.Tools.pandaTools import initMetadata
import pandas as pd
import numpy as np
import math
import logging

x bokehVisJS3DGraph.ts
x HistogramCDS.ts
Import  CDSCompress.ts
Import  DownsamplerCDS.ts
Import  CDSAlias.ts
Import  CDSJoin.ts


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### Read clusters csv file

In [3]:
path = "../output/"
file = "run456060805_221112060811_clusters.csv"
df = pd.read_csv(path+file)
initMetadata(df)
df.meta.metaData = {    
    "localX.AxisTitle":      "local X (mm)",
    "localY.AxisTitle":      "local Y (mm)",
    "globalX.AxisTitle":     "global X (mm)",
    "globalY.AxisTitle":     "global Y (mm)",
    "globalZ.AxisTitle":     "global Z (mm)",
    "ID.AxisTitle":          "ALPIDE",
    "nPixels.AxisTitle":     "Number of pixels in cluster",
    "columnWidth.AxisTitle": "Cluster width in columns (Width in X)",
    "rowWidth.AxisTitle": "Cluster width in rows (Width in Y)"
}

In [4]:
myMap = {"ALPIDE_0":0,"ALPIDE_1":1,"ALPIDE_2":2,"ALPIDE_3":3,"ALPIDE_4":4}
df["ID"] = df["detector"].map(myMap)
variables=list([i for i in df.columns if "detector" not in i])
df.head()

,eventNr,detector,localX,localY,globalX,globalY,globalZ,charge,split,nPixels,columnWidth,rowWidth,timeStamp,ID
0,0,ALPIDE_0,5.21934,-6.06144,-29.5471,-5.19305,6.43856,1,0,1,1,1,6.456710e+14,0
1,0,ALPIDE_0,12.09070,-5.52384,-27.5964,-11.76610,6.97616,1,0,1,1,1,6.456710e+14,0
2,40,ALPIDE_4,8.99130,-5.57760,28.6627,8.85729,6.92240,1,0,1,1,1,6.467850e+14,4
3,48,ALPIDE_1,-8.02053,0.95424,-22.6722,7.87207,7.20424,10,0,10,4,3,6.469990e+14,1
4,48,ALPIDE_2,-7.53196,3.01423,-16.4470,7.31407,3.01423,11,0,11,3,4,6.470000e+14,2


In [5]:
aliasArray = [
    #("ALPIDE_bitmap","0x1*(detector==\"ALPIDE_0\")")
]

aliasArray+=[
    {
        "name": "customFunction",
        "variables": [i for i in variables if  "ustom" not in i ],
        "func":"customFunction",
    }
]

### Define dashboard

In [6]:
axisTransforms = [None, "sqrt", "lambda x: log(1+x)","lambda x: 1/sqrt(x)"]

parameterArray = [
    # Parameters to enable custom histogramming
    {"name": "varX", "value":"localX", "options":variables},
    {"name": "varY", "value":"localY", "options":variables},
    {"name": "varZ", "value":"nPixels", "options":variables},
    {"name": "nbinsX", "value":30, "range":[10, 200]},
    {"name": "nbinsY", "value":30, "range":[10, 200]},
    {"name": "nbinsZ", "value":5, "range":[1,10]},
    {"name": "varColor", "value":"nPixels", "options":variables},
    # Transformations on axes
    {'name': "xAxisTransform", "value":None, "options":axisTransforms},
    {'name': "yAxisTransform", "value":None, "options":axisTransforms},
    {'name': "zAxisTransform", "value":None, "options":axisTransforms},
    # custom selection
    {'name': 'customFunction', "value":"return 1"},
]

parameterArray.extend(figureParameters["legend"]['parameterArray'])   
parameterArray.extend(figureParameters["markers"]['parameterArray'])  

In [7]:
widgetParams = [
    # Selection on data
    ["range", ["localX"]],
    ["range", ["localY"]],
    ["range", ["globalX"]],
    ["range", ["globalY"]],
    ["range", ["globalZ"]],
    ["multiSelect", ["detector"]],
    ["range", ["eventNr"]],
    ["range", ["timeStamp"]],
    ["range", ["charge"]],
    ["range", ["nPixels"]],
    ["range", ["columnWidth"]],
    ["range", ["rowWidth"]],
    ["multiSelect", ["split"]],
    ['text', ['customFunction'], {"name": "Custom selection"}],
    # Selection for visualization
    ['select', ['varX'], {"name": "varX"}],
    ['select', ['varY'], {"name": "varY"}],
    ['select', ['varZ'], {"name": "varZ"}],
    ['slider', ['nbinsX'], {"name": "nbinsX"}],
    ['slider', ['nbinsY'], {"name": "nbinsY"}],
    ['slider', ['nbinsZ'], {"name": "nbinsZ"}],
    ['select', ['xAxisTransform'], {"name": "xAxisTransform"}],
    ['select', ['yAxisTransform'], {"name": "yAxisTransform"}],
    ['select', ['zAxisTransform'], {"name": "zAxisTransform"}],
    ['select', ['varColor'], {"name": "varColor"}],
]

widgetParams.extend(figureParameters["legend"]["widgets"])
widgetParams.extend(figureParameters["markers"]["widgets"])

widgetLayoutDesc = {
    "Selection": [[0,1,2,3,4],[5,6,7,8],[9,10,11,12],[13],{"sizing_mode": "scale_width"}],
    "Plotting" : [[14,15,16],[17,18,19],[20,21,22],[23],{"sizing_mode": "scale_width"}],
    "Legend": figureParameters['legend']['widgetLayout'],
    "Markers":["markerSize"]
} 

figureGlobalOption={}
figureGlobalOption=figureParameters["legend"]["figureOptions"]
figureGlobalOption["size"]="markerSize"
figureGlobalOption["x_transform"]="xAxisTransform"
figureGlobalOption["y_transform"]="yAxisTransform"
figureGlobalOption["z_transform"]="zAxisTransform"

In [8]:
histoArray = [
        {"name": "histo2Dhitmap", "variables": ["localX","localY"], "nbins":["nbinsX","nbinsY"], "axis":[1],"unbinned_projections":True},
        {"name": "histo3Dcustom", "variables": ["varX","varY","varZ"], "nbins":["nbinsX","nbinsY","nbinsZ"],"axis":[1,2],"unbinned_projections":True},
        {"name": "histo2Dcustom", "variables": ["varX","varY"], "nbins":["nbinsX","nbinsY"], "axis":[1],"unbinned_projections":True},
        {"name": "histo1Dcustom", "variables": ["varX"], "nbins":"nbinsX"},

    ]

figureArray = [
    # 3D plot
    [["globalZ"], ["globalX"], {"color": "red", "size": 2, "varZ": "globalY", "colorZvar": "varColor", "name":"3D"}],
    # Hitmap
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar":"bin_count","source":"histo2Dhitmap","name":"2Dhitmap"}],
    # Custom histogramming
    [["bin_center"],["bin_count"],{"source":"histo1Dcustom"}],
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar":"bin_count","source":"histo2Dcustom"}],
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar": "mean", "source":"histo3Dcustom_2"}],
    #
    figureGlobalOption
]

In [9]:
layout = {
    "Geometry" : [[0],{"plot_height":500,"sizing_mode":"fixed"}],
    "Hitmap" : [[1],{"plot_height":500,"sizing_mode":"fixed"}],
    "Custom" : [[2,3,4],{"plot_height":500}]
}

In [10]:
tooltips = []
output_file("../output/"+file[0:-4]+".html")
arrayCompressionRelative10=[(".*",[("relative",10), ("code",0), ("zip",0), ("base64",0)])]
arrayCompression=arrayCompressionRelative10
bokehDrawSA.fromArray(df, "eventNr>=0", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                      histogramArray=histoArray,aliasArray=aliasArray,arrayCompression=arrayCompression,parameterArray=parameterArray,
                      sizing_mode='scale_width',rescaleColorMapper=True,)

/home/berki/Software/RootInteractive/RootInteractive/InteractiveDrawing/bokeh/bokehTools.py:1447: FutureWarning: Specifying `na_sentinel=None` is deprecated, specify `use_na_sentinel=False` instead.
  codes, optionsPlot = pd.factorize(dfCategorical, sort=True, na_sentinel=None)


compressCDSPipe
Compress 1 globalZ .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 1406130 11982579 0.11734785975539991 1 globalZ
Compress 2 globalX .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 956922 11982579 0.07985943593612026 2 globalX
Compress 3 globalY .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 1393714 11982579 0.11631168882758879 3 globalY
Compress 4 nPixels .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 545602 9985603 0.054638863571884445 4 nPixels
Compress 5 charge .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 545602 9985602 0.05463886904364905 5 charge
Compress 6 localX .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 1390154 11982578 0.11601460053087073 6 localX
Compress 7 rowWidth .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 346418 9985604 0.0346917422321

Compress 11 localY .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 1383926 11982578 0.11549484593382159 11 localY
Compress 12 timeStamp .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 2826 11982581 0.00023584234481703066 12 timeStamp
Compress 13 eventNr .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 14018 9985603 0.001403821081210619 13 eventNr
Compress 14 detector.factor() .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 115502 1997138 0.05783376011071844 14 detector.factor()
Compress 15 split.factor() .* [('relative', 10), ('code', 0), ('zip', 0), ('base64', 0)]
Compress factor 2814 1997138 0.0014090163023286323 15 split.factor()
Compress _all 8544226 145788968 0.058606807615237386 15
